# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Lab for basic text operation with Gemini and understanding Gemini fundamental.


## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.2 MB/s eta 0:00:00


## Authentication to access to the GCP & Google drive

In [2]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()


## Set the environment on GCP Project

In [3]:
# Set the environment on GCP Project
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"
MODEL_NAME="gemini-1.5-flash"

## Vertex AI initialization


In [4]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
)

# Initialize the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION) # https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

system_instructions = [
 "Respond in Korean",
 "Respond in structured ways"
]

# Access to the generative model.
# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

## Function to get the response

In [5]:
def generate(query:str, stream:bool):
    """
    Generate a response from the model.
    query :query to be sent to the model
    Returns:The generated response.
    """

    # Request body : https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#request
    # parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = GenerationConfig(
        temperature=0.9,
        top_p=1.0,
        top_k=32,
        candidate_count=1,
        max_output_tokens=8192,
    )

    contents = [query]

    # Counts tokens
    print(f"Token Count : {model.count_tokens(contents)}")

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    }

    responses = model.generate_content(
        contents,
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=stream,
    )

    return responses



## Run example

In [6]:

from time import perf_counter

t1_start = perf_counter()

prompt = """
          You are a Generative AI specialist to help people understand the concept of AI.
          Explain what's the pros and cons of using AI in the following instructions.

          1. Define what's the Generative AI.
          2. What are differences between typical machine learning and Generative AI.
          3. What are the pros and cons of using Generative AI.
          4. Explain use cases of Generative AI in the manufacturing industry.
"""
stream = False

responses = generate(prompt, stream)

if stream:
  for response in responses:
    print(response.text, end="")
else:
  # Print the model response
  print(f'\n Usage metadata:\n{responses.to_dict().get("usage_metadata")}')
  print(f"\n Finish reason:\n{responses.candidates[0].finish_reason}")
  print(f"\n Safety settings:\n{responses.candidates[0].safety_ratings}")

  display(Markdown(responses.text))

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")



Token Count : total_tokens: 105
total_billable_characters: 363


 Usage metadata:
{'prompt_token_count': 105, 'candidates_token_count': 698, 'total_token_count': 803}

 Finish reason:
1

 Safety settings:
[category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
probability_score: 0.0260009766
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0942382812
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
probability_score: 0.0267333984
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.135742188
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
probability_score: 0.0981445312
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0849609375
, category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
probability_score: 0.0395507812
severity: HARM_SEVERITY_NEGLIGIBLE
severity_score: 0.0559082031
]


## 생성형 AI에 대한 안내: 장단점과 제조업 적용 사례

**1. 생성형 AI란 무엇인가요?**

생성형 AI는 새로운 콘텐츠를 생성하는 데 특화된 인공지능의 한 유형입니다. 텍스트, 이미지, 오디오, 비디오, 코드와 같은 다양한 형태의 콘텐츠를 생성할 수 있으며, 이는 기존 데이터를 학습하고 새로운 데이터를 생성하는 데 사용됩니다.

**2. 일반적인 머신러닝과 생성형 AI의 차이점은 무엇인가요?**

* **일반적인 머신러닝:** 기존 데이터를 기반으로 미래를 예측하거나 분류하는 데 중점을 둡니다. 예를 들어, 이메일 스팸 필터링, 이미지 분류, 주식 시장 예측 등이 있습니다.
* **생성형 AI:** 새로운 콘텐츠를 생성하는 데 중점을 둡니다. 예를 들어, 텍스트 생성, 이미지 생성, 음악 작곡, 3D 모델링 등이 있습니다.

**3. 생성형 AI 사용의 장단점은 무엇인가요?**

**장점:**

* **창의성 향상:** 새로운 아이디어와 콘텐츠를 생성하여 창의성을 확대할 수 있습니다.
* **생산성 향상:** 반복적인 작업을 자동화하여 생산성을 높일 수 있습니다.
* **개인화된 경험 제공:** 사용자의 선호도에 맞춰 콘텐츠를 생성하여 개인화된 경험을 제공할 수 있습니다.

**단점:**

* **데이터 편향 문제:** 학습 데이터에 존재하는 편향이 생성된 콘텐츠에 반영될 수 있습니다.
* **저작권 문제:** 생성된 콘텐츠의 저작권 소유 및 사용에 대한 명확한 기준이 부족할 수 있습니다.
* **신뢰성 문제:** 생성된 콘텐츠의 정확성과 신뢰성을 검증하기 어려울 수 있습니다.

**4. 제조업에서의 생성형 AI 활용 사례는 무엇인가요?**

* **설계 최적화:** 생성형 AI를 활용하여 제품 설계를 최적화하고, 새로운 디자인을 생성할 수 있습니다.
* **생산 계획 수립:** 생산 일정, 자재 소요량 등을 예측하고 계획을 수립하는 데 활용할 수 있습니다.
* **품질 관리:** 제품 불량률을 예측하고 품질 관리 프로세스를 개선하는 데 도움이 될 수 있습니다.
* **예지 정비:** 장비 고장을 예측하고 사전에 유지보수를 수행하여 생산 중단을 방지할 수 있습니다.

**결론:**

생성형 AI는 혁신적인 기술이지만, 사용 전에 장단점을 신중하게 고려해야 합니다. 적절한 데이터 전처리, 편향 해소, 윤리적 문제 해결 등을 통해 생성형 AI를 안전하고 효과적으로 활용할 수 있도록 노력해야 합니다. 


Time : 6.175823661999971 seconds


